#### *THIS NOTEBOOK MEANT TO RUN ON RSP*

This notebook loads cModel and GaaP photometry for all "extended" DP1 objects that have SNR > 10 in the $i$ band and fainter than 20th mag. The fluxes are de-reddened using the SFD map, and then cross-matched with photo-z's using the object IDs.

In [ ]:
pwd

### 1. Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, join

from lsst.rsp import get_tap_service

### 2. Query galaxies from the TAP

In [ ]:
service = get_tap_service("tap")

In [ ]:
query = """
SELECT
    obj.objectID,
    obj.coord_ra,
    obj.coord_dec,
    obj.u_cModelFlux,
    obj.u_cModelFluxErr,
    obj.g_cModelFlux,
    obj.g_cModelFluxErr,
    obj.r_cModelFlux,
    obj.r_cModelFluxErr,
    obj.i_cModelFlux,
    obj.i_cModelFluxErr,
    obj.z_cModelFlux,
    obj.z_cModelFluxErr,
    obj.y_cModelFlux,
    obj.y_cModelFluxErr,
    obj.u_gaap1p0Flux,
    obj.u_gaap1p0FluxErr,
    obj.g_gaap1p0Flux,
    obj.g_gaap1p0FluxErr,
    obj.r_gaap1p0Flux,
    obj.r_gaap1p0FluxErr,
    obj.i_gaap1p0Flux,
    obj.i_gaap1p0FluxErr,
    obj.z_gaap1p0Flux,
    obj.z_gaap1p0FluxErr,
    obj.y_gaap1p0Flux,
    obj.y_gaap1p0FluxErr,
    obj.ebv,
    obj.i_hsmShapeRegauss_e1,
    obj.i_hsmShapeRegauss_e2,
    obj.i_hsmShapeRegauss_flag,
    obj.i_hsmShapeRegauss_sigma,
    obj.i_blendedness,
    obj.i_iPSF_flag,
    obj.i_ixxPSF,
    obj.i_iyyPSF,
    obj.i_ixx,
    obj.i_iyy
FROM dp1.Object AS obj
WHERE
    obj.i_cModelFlux / obj.i_cModelFluxErr > 10 AND
    scisql_nanojanskyToAbMag(obj.i_cModelFlux) > 20 AND
    obj.i_cModel_flag = 0 AND
    obj.r_cModel_flag = 0 AND
    obj.i_cModelFlux > 0 AND
    obj.r_cModelFlux > 0 AND
    obj.i_extendedness > 0.5 AND
    obj.r_extendedness > 0.5
"""
# If you want to only select ECDFS
# CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), CIRCLE('ICRS', 53.1, -28.1, 0.75)) = 1

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=["COMPLETED", "ERROR"])
print("Job phase is", job.phase)

In [ ]:
results = job.fetch_result()
cat = results.to_table()
cat

### 3. Deredden the fluxes

In [ ]:
# EBV coefficients
band_a_ebv = dict(
    u=4.81,
    g=3.64,
    r=2.70,
    i=2.06,
    z=1.58,
    y=1.31,
)

for band in "ugrizy":
    # Load the EBV coefficient
    a_ebv = band_a_ebv[band]

    # Loop over every flux type
    for flux_type in ["cModel", "gaap1p0"]:
        # Get the name of the column
        col = f"{band}_{flux_type}Flux"

        # De-redden
        cat[col] = cat[col] * 10 ** (a_ebv * cat["ebv"] / 2.5)

### 4. Determine shear cuts

Using cuts from Table 4 of https://arxiv.org/pdf/1705.06745

In [ ]:
# For shear sample
e = np.sqrt(cat["i_hsmShapeRegauss_e1"] ** 2 + cat["i_hsmShapeRegauss_e2"] ** 2)
R2 = 1 - (cat["i_ixxPSF"] + cat["i_iyyPSF"]) / (cat["i_ixx"] + cat["i_iyy"])

shear_cut = (
    (e < 2)  # Remove spurious shapes
    & (cat["i_hsmShapeRegauss_sigma"] <= 0.4)  # Shape uncertainty not too high
    & (cat["i_hsmShapeRegauss_flag"] == 0)  # Shape measurement succeeded
    & (cat["i_blendedness"] < 0.42)  # Object not too blended
    & (cat["i_iPSF_flag"] == 0)  # Successfully measured moments of PSF
    & (R2 >= 0.3)  # Galaxy sufficiently resolved (removes small artifacts)
)
cat["use_shear"] = shear_cut.data

# Estimate shear from shapes
R = 0.85  # Shear responsivity for HSM Regaussianization; typical value from HSC
cat["shear_g1"] = cat["i_hsmShapeRegauss_e1"] / (2 * R)
cat["shear_g2"] = cat["i_hsmShapeRegauss_e2"] / (2 * R)

# Save info related to shear uncertainties
i_snr = cat["i_cModelFlux"] / cat["i_cModelFluxErr"]
shape_noise = 0.264 * (i_snr / 20) ** -0.891 * (R2 / 0.5) ** -1.015
measurement_err = 0.4
cat["shear_err"] = np.sqrt(shape_noise**2 + measurement_err**2)

### 5. Cross-match with photo-z's

In [ ]:
pz = Table.read("dp1_photoz_no_mags.parquet")

In [ ]:
combined = join(cat, pz, keys_left="objectID", keys_right="objectId")

In [ ]:
combined.write("dp1_with_photoz_v2.fits")

In [ ]:
cat